In [1]:
import numpy as np
import pandas as pd

In [2]:
feed_info = pd.read_csv('E:/wechat_algo_data/feed_info.csv')
feed_info.head(3)

,feedid,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char
0,43549,6165,38,104741 122649 8109 117252 65632 23463 118668 4...,139499 59421 82007 142955 27736 83577 52394 11...,142955 27736 83577 103956 32010 34170 89740 90...,19356.0,11703.0,15506;7715;17582,26334;219;25209;7715;18541,81;269;159;6,269 0.8525666;81 0.8525666;8 1.1e-07;306 0.0;2...,26439 5247 6426 3827 1882 26018 20744 22204 30...,25926 8491 13394 2203 26439 6243 33054 16435 1...,2203 26439 6243 33054 16435 16307 17070 24908 ...
1,77432,9386,60,35753 27736 146603 73055 11794 101761 11794 81...,35753 146603 73055 11794 101761 67496 16933 52...,146739 14368 79290 79213 47366 8109 33194 1198...,NaN,NaN,8199;18322;4243,24078;19924,194;267;159;6,267 0.99293476;194 0.99293476,31010 32495 6243 13923 15360 30483 2709 26084 ...,31010 32495 13923 15360 30483 2709 26084 15160...,7259 20851 5061 26207 17573 17531 15117 20072 ...
2,12921,2996,15,114413 107973 117252 27736 41035 32715 125374 ...,NaN,NaN,10288.0,11354.0,NaN,11696;6926,202;23;160;6,23 0.64771646;202 0.64771646,27077 10050 27464 20072 26018 6243 21492 14750...,NaN,NaN


In [3]:
feed_info.query('authorid == 9386').shape

(6, 15)

In [4]:
feed_info = feed_info.loc[:100000]

In [5]:
feed_info.shape

(100001, 15)

In [6]:
# action数据
action_df = pd.read_csv('E:/wechat_algo_data/user_action.csv')
action_df.head(3)

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite
0,8,71474,1,1,0,0,1,500,5366,0,0,0,0
1,8,73916,1,1,0,0,0,250,1533,0,0,0,0
2,8,50282,1,1,0,0,0,750,1302,0,0,0,0


In [28]:
# 预测数据
test = pd.read_csv('E:/wechat_algo_data/test_a.csv')
test.head(3)

,userid,feedid,device
0,14298,67227,1
1,68356,91864,2
2,49925,104657,2


In [7]:
import os
import copy
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [8]:
# 对list字段进行切分和映射编码
#key2index就是将每个唯一的键映射到一个唯一的整数索引。NLP
def split(column):
    if not isinstance(column,str):
        return []
    keys = column.strip().split(';')
    for key in keys:
        if key not in key2index:
            key2index[key] = len(key2index) + 1 #将每个唯一的 key 映射到一个唯一的索引值
    return list(map(lambda x: key2index[x], keys))

In [9]:
def preprocess(sample,dense_features):
    '''
    特征工程：对数值型特征做对数变换; id型特征+1; 缺失值补充0。
    '''
    sample[dense_features] = sample[dense_features].fillna(0.0)
    sample[dense_features] = np.log(sample[dense_features] + 1.0)
    
    sample[["authorid", "bgm_song_id", "bgm_singer_id"]] += 1  # 0 用于填未知
    sample[["authorid", "bgm_song_id", "bgm_singer_id", "videoplayseconds"]] = sample[["authorid", "bgm_song_id", "bgm_singer_id", "videoplayseconds"]].fillna(0)
    sample["videoplayseconds"] = np.log(sample["videoplayseconds"] + 1.0)
    sample[["authorid", "bgm_song_id", "bgm_singer_id"]] = sample[["authorid", "bgm_song_id", "bgm_singer_id"]].astype(int)
    return sample

In [10]:
action_df.sample(frac = 1.0)

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite
6697675,229500,56135,4,2,0,0,0,17255,17609,0,0,0,0
5331353,182503,52199,11,2,0,0,0,2230,2440,0,0,0,0
687340,23239,29558,9,2,0,0,0,28888,28972,0,0,0,0
3987956,136169,109161,13,2,0,0,0,27828,28714,0,0,0,0
519829,17676,33478,3,1,0,0,0,13750,16046,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4916309,168346,57239,7,2,0,0,0,49164,50469,0,0,0,0
2322017,78288,93147,1,2,0,0,0,7623,7821,0,0,0,0
104136,3538,4739,4,2,0,0,0,111,509,0,0,0,0
2723704,92876,109441,3,2,0,0,0,38629,38647,0,0,0,0


In [11]:
action_df = action_df.loc[:600000]

In [34]:
test['date_'] = 15
action_df = pd.concat([action_df,test])

In [12]:
action_df.query('date_ == 15')

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite


In [13]:
# 标签列
target = ["read_comment", "like", "click_avatar", "forward"]
# 稀疏特征
sparse_features = ['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id']
# 变长序列特征
varlen_features = ['manual_tag_list','manual_keyword_list']
# 稠密特征
dense_features = ['videoplayseconds']

In [14]:
# 数据合并
feed_info = feed_info[['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id','manual_tag_list','manual_keyword_list']]
data = action_df.merge(feed_info, how='left',on='feedid') #行为数据拼接，作者id，bgm_song_id 
data = preprocess(data,dense_features) #特征处理
data = data[dense_features+sparse_features+varlen_features+['date_']+target]

In [15]:
data

,videoplayseconds,userid,feedid,authorid,bgm_song_id,bgm_singer_id,manual_tag_list,manual_keyword_list,date_,read_comment,like,click_avatar,forward
0,1.248441,8,71474,1529,13746,3557,219;6;124,NaN,1,0,1,0,0
1,1.343703,8,73916,1443,0,0,308;6,NaN,1,0,0,0,0
2,1.496434,8,50282,8649,0,0,9;6;222;222,NaN,1,0,0,0,0
3,1.080418,8,11391,11977,13097,5013,228;12;159;6,24390;14546;6355,1,0,1,0,0
4,1.271150,8,27349,4371,22216,7900,322;13;159;6,9868;27100;11411,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
599996,1.448822,20339,56221,18465,14915,10420,9;6,NaN,9,0,0,0,0
599997,1.439569,20339,69673,14651,0,0,332;16;159;6,26617;208;26921;16061,9,0,0,0,0
599998,1.621454,20339,85573,6584,14417,6285,332;16;159;6,26617;23423;20219,9,0,0,0,0
599999,1.631370,20339,6842,2537,1698,11753,29;271;159;6,9165;26054,9,0,0,0,0


In [16]:
# 变长特征编码
encoder = {}
global key2index
for f in ['manual_keyword_list','manual_tag_list']:
    key2index = {}
    f_list = list(map(split, data[f].values))#把split函数用到每个values里
    f_length = np.array(list(map(len, f_list)))#把len函数用到每个list里 
    max_len = max(f_length)
    print(f'{f} 字段最长的取值序列长度为 {max_len}')
    # Notice : padding=`post`
    data[f] = list(pad_sequences(f_list, maxlen=max_len, padding='post', ))
    encoder[f] = copy.copy(key2index)

manual_keyword_list 字段最长的取值序列长度为 18
manual_tag_list 字段最长的取值序列长度为 11


In [17]:
f_list

[[1, 2, 3],
 [4, 2],
 [5, 2, 6, 6],
 [7, 8, 9, 2],
 [10, 11, 9, 2],
 [12, 13, 9, 2],
 [14, 15, 9, 16, 17],
 [18, 19, 20, 9, 2],
 [21, 22, 9, 2],
 [12, 13, 9, 2],
 [1, 2],
 [5, 2],
 [12, 13, 9, 2],
 [7, 8, 9, 2],
 [5, 2],
 [5, 2],
 [5, 2, 3],
 [12, 13, 9, 2],
 [18, 19, 9, 2],
 [2],
 [18, 19, 23, 2],
 [5, 2],
 [18, 19, 9, 2],
 [12, 13, 9, 2],
 [24, 25, 23, 2],
 [26, 27, 9, 2],
 [1, 2],
 [14, 15, 9, 16],
 [18, 19, 9, 28, 2],
 [12, 13, 9, 2],
 [29, 30, 9, 2],
 [12, 13, 9, 2],
 [1, 2],
 [18, 19, 9, 28, 2],
 [31, 32, 9, 33, 2],
 [31, 32, 9, 33, 2],
 [18, 19, 9, 28, 2],
 [1, 2],
 [34, 35, 23, 2],
 [14, 15, 9, 16],
 [36, 19, 23, 2],
 [18, 19, 9, 28, 2],
 [37, 38, 9, 2, 3],
 [39, 32, 9, 33, 2],
 [12, 13, 9, 2],
 [18, 19, 9, 40, 2],
 [18, 19, 9, 28, 20, 2],
 [41, 25, 9, 2],
 [7, 8, 9, 2],
 [10, 11, 9, 2],
 [1, 2, 3, 3],
 [42, 2],
 [5, 2, 3, 43],
 [12, 13, 9, 2],
 [18, 19, 23, 20, 2],
 [12, 13, 9, 2],
 [18, 19, 28, 9, 20, 2],
 [44, 32, 9, 2],
 [45, 35, 9, 2],
 [7, 8, 9, 2],
 [21, 22, 9, 46, 2],
 

In [18]:
# 稀疏特征编码
for featid in sparse_features:
    print(f"编码ID字段：{featid}")
    encoder[featid] = {uid:ucode+1 for ucode,uid in enumerate(data[featid].unique())} #返回一个map
    data[featid] = data[featid].apply(lambda x: encoder[featid].get(x,0))#有返回x，无返回0

编码ID字段：userid
编码ID字段：feedid
编码ID字段：authorid
编码ID字段：bgm_song_id
编码ID字段：bgm_singer_id


In [ ]:
# print('数据维度：', data.shape)
# print('数据字段：', data.columns.tolist())
# print('不同的date_取值: ', data['date_'].unique())
# # 如果资源比较少，可以在这里进行数据采样
# temp = data.sample(frac = 1.0)#随机排序，降低拟合

数据维度： (600001, 13)
数据字段： ['videoplayseconds', 'userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id', 'manual_tag_list', 'manual_keyword_list', 'date_', 'read_comment', 'like', 'click_avatar', 'forward']
不同的date_取值:  [ 1  2  3  5  6  7  8 10 11 12 13 14  4  9]


In [21]:
data

,videoplayseconds,userid,feedid,authorid,bgm_song_id,bgm_singer_id,manual_tag_list,manual_keyword_list,date_,read_comment,like,click_avatar,forward
0,1.248441,1,1,1,1,1,"[1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,1,0,0
1,1.343703,1,2,2,2,2,"[4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0
2,1.496434,1,3,3,2,2,"[5, 2, 6, 6, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0
3,1.080418,1,4,4,3,3,"[7, 8, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,1,0,0
4,1.271150,1,5,5,4,4,"[10, 11, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
599996,1.448822,1643,17468,6056,1272,1087,"[5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9,0,0,0,0
599997,1.439569,1643,28561,3122,2,2,"[34, 35, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[386, 62, 1319, 708, 0, 0, 0, 0, 0, 0, 0, 0, 0...",9,0,0,0,0
599998,1.621454,1643,7179,434,2648,2193,"[34, 35, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[386, 230, 114, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9,0,0,0,0
599999,1.631370,1643,17421,6045,5469,4326,"[137, 25, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[179, 164, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",9,0,0,0,0


In [44]:
data.query('videoplayseconds > 1')

,videoplayseconds,userid,feedid,authorid,bgm_song_id,bgm_singer_id,manual_tag_list,manual_keyword_list,date_,read_comment,like,click_avatar,forward
156837,1.631370,442,27824,8187,2,2,"[74, 72, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[8213, 182, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",3,0.0,0.0,0.0,0.0
43856,1.310506,127,21089,6906,2,2,"[157, 91, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[342, 6, 25, 14, 6668, 0, 0, 0, 0, 0, 0, 0, 0,...",8,0.0,0.0,0.0,0.0
74585,1.358505,211,9911,464,3516,439,"[71, 72, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[170, 171, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,0.0,0.0,0.0,0.0
997000,1.194706,4999,2181,1297,866,759,"[5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",15,NaN,NaN,NaN,NaN
675170,1.509918,7614,20942,2560,2,2,"[31, 32, 9, 48, 33, 2, 0, 0, 0, 0, 0]","[40, 21, 1197, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425496,1.429846,1164,8412,1703,655,589,"[5, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6,0.0,0.0,0.0,0.0
371754,1.631370,1018,34158,9224,2,2,"[87, 55, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[271, 9516, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",12,0.0,0.0,0.0,0.0
890295,1.631370,4591,46426,11027,9548,2063,"[241, 59, 9, 2, 0, 0, 0, 0, 0, 0, 0]","[1745, 62, 108, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",15,NaN,NaN,NaN,NaN
667407,1.429846,452,176,139,2,2,"[14, 15, 9, 16, 0, 0, 0, 0, 0, 0, 0]","[10, 135, 9, 136, 163, 0, 0, 0, 0, 0, 0, 0, 0,...",15,NaN,NaN,NaN,NaN


In [45]:
!mkdir data_and_feature

子目录或文件 data_and_feature 已经存在。


In [46]:
# 构建训练集，验证集和测试集
# 第14天样本作为验证集
train = data[data['date_'] < 14].drop(['date_'],axis = 1)
val = data[data['date_'] == 14].drop(['date_'],axis = 1)
test = data[data['date_'] == 15].drop(['date_'],axis = 1)

In [47]:
alldata = data[data['date_'] <= 14].drop(['date_'],axis = 1)

In [22]:
alldata = data

In [24]:
import gc
import joblib

del feed_info
del data
gc.collect()

0

In [25]:
# joblib.dump(train, './data_and_feature/train.txt')
# joblib.dump(val, './data_and_feature/val.txt')
# joblib.dump(test, './data_and_feature/test.txt')
joblib.dump(encoder, './data_and_feature/encoder.txt')
joblib.dump(alldata, './data_and_feature/alldata.txt')

['./data_and_feature/alldata.txt']